#Format Data for Experiment in ABM environment

In [1]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl
import itertools
from collections import Counter

In [2]:
#Import other needed packages
from vivarium.library.units import units, remove_units
from tumor_tcell.experiments.main import tumor_tcell_abm
from tumor_tcell.experiments.main import plots_suite
from tumor_tcell.experiments.main import get_tcells
from tumor_tcell.experiments.main import get_tumors
from tumor_tcell.library.phylogeny import get_phylogeny

Loading chipmunk for Linux (64bit) [/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/vivarium_env/lib/python3.8/site-packages/pymunk/libchipmunk.so]


In [3]:
#Read in the data from parent directory
analysis_dir = '/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/'
experiment_id = '2HC/'
experiment_dir = analysis_dir+experiment_id
os.chdir(experiment_dir)
os.getcwd()

'/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/2HC'

In [4]:
#Read in the data from parent directory
df_2HC_exp = pd.read_csv('2HCTcell_exp_1of4.csv')
df_2HC_exp.rename(columns = {"Unnamed: 0": "cell_index"},  inplace = True) 
df_2HC_exp.set_index('cell_index', inplace = True)
df_2HC_exp

,AbsoluteY,AbsoluteX,H2Db,H2Kb,PD1,PDL1,Region,Cell Type,IFNg
cell_index,,,,,,,,,
1274,31.562030,138.168682,7110.647333,6511.161015,2632.219092,15436.683863,2HCTcell_d3,PDL1n,0
1276,38.644678,99.184112,9882.720149,8936.648009,2849.635188,23856.816070,2HCTcell_d3,PDL1n,0
1278,39.749814,90.089597,6811.345951,10995.800411,3264.648695,16945.290495,2HCTcell_d3,PDL1n,0
1282,47.158012,247.362093,3675.514222,1964.683229,2033.355383,4849.012421,2HCTcell_d3,PDL1n,0
1285,8.397423,29.765645,5933.431247,7993.782521,2901.706926,19047.454667,2HCTcell_d3,PDL1n,0
...,...,...,...,...,...,...,...,...,...
17640,685.217626,718.389745,11528.709624,25363.626005,13704.653409,23124.886773,2HCTcell_d3,PD1n,0
17641,631.483713,583.280714,40704.676698,38641.812327,4985.192130,34429.775836,2HCTcell_d3,PD1n,0
17642,762.779854,847.480997,9087.268135,34835.322826,7723.293285,29137.886824,2HCTcell_d3,PD1n,0


In [5]:
#Set bounds
bounds = [1800,1800]

#Create a copy
df_2HC_exp1 = df_2HC_exp.copy()

#Column Number for iloc iterating
Ypos = 0
Xpos = 1
H2Kb = 3
PD1 = 4
PDL1 = 5
Celltype = 7
IFNg_con = 8 

tumor_diameter = 15
tcell_diameter = 7.5

tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_2HC_exp1)):
  if df_2HC_exp1.iloc[i, Celltype] == 'PDL1p' or df_2HC_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PDL1': df_2HC_exp1.iloc[i, PDL1],
            'MHCI': df_2HC_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })
    
    IFNg.update({
        df_2HC_exp1.index[i] : {
            'amount': df_2HC_exp1.iloc[i, IFNg_con],
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else: 
    tcells.update({
        'tcell_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PD1': df_2HC_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_2HC_exp1.index[i]: {
        'amount': df_2HC_exp1.iloc[i, IFNg_con],
        'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

tcells

{'tcell_17561': {'location': [115.20390805053712 <Unit('micrometer')>,
   489.4741821289063 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 33259.25807056563,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17562': {'location': [133.49253839111327 <Unit('micrometer')>,
   474.6180258789064 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 39453.705806085374,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17564': {'location': [260.4977958984375 <Unit('micrometer')>,
   413.5967653808593 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 27198.82947357501,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17565': {'location': [452.20804846191413 <Unit('micrometer')>,
   552.7608481445313 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 19922.6654816716,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17566': {'location': [474.46803002929687 <Unit('micrometer')>,
   603.99879296875 <Un

#Run tumor-tcell ABM Experiment

In [ ]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors, 
                       tcells=tcells, 
                       total_time=259200, 
                       bounds=[b*units.um for b in bounds], 
                       halt_threshold=10000,
                       emit_step=600,
                       emitter='database',)
data;

Initializing experiment tumor_tcell_20210508.175135


  1%|          | 3/432 [06:05<14:29:17, 121.58s/it]